In [146]:
from dotenv import load_dotenv
import gradio as gr
import requests
import json
import os

In [126]:
load_dotenv(override=True)

True

In [128]:
from openai import OpenAI

In [129]:
openai = OpenAI()

In [145]:
global_name = ""

In [131]:
record_user_interests_details_json = {
    "name": "record_user_interests_details",
    "description": "Use this tool to record the interests of the user and their notification schedule",
    "parameters": {
        "type": "object",
        "properties": {
            "category": {
                "type": "object",
                "description": "The category the user is interested in being notified about.",
                "properties": {
                    "interest": {
                        "type": "string",
                        "description": "The interests of the user"
                    },
                    "description": {
                        "type": "string",
                        "description": "additional details about users interests"
                    }
                }
            },
            "name": {
                "type": "string",
                "description": "The name of the user"
            },
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "phone": {
                "type": "string",
                "description": "The phone number of this user with country code"
            },
        }
    },
    "required": ["email"],
    "additionalProperties": False
}

In [132]:
tools = [{"type": "function", "function": record_user_interests_details_json}]

In [133]:
tools

[{'type': 'function',
  'function': {'name': 'record_user_interests_details',
   'description': 'Use this tool to record the interests of the user and their notification schedule',
   'parameters': {'type': 'object',
    'properties': {'category': {'type': 'object',
      'description': 'The category the user is interested in being notified about.',
      'properties': {'interest': {'type': 'string',
        'description': 'The interests of the user'},
       'description': {'type': 'string',
        'description': 'additional details about users interests'}}},
     'name': {'type': 'string', 'description': 'The name of the user'},
     'email': {'type': 'string',
      'description': 'The email address of this user'},
     'phone': {'type': 'string',
      'description': 'The phone number of this user with country code'}}},
   'required': ['email'],
   'additionalProperties': False}}]

In [134]:
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [135]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [143]:
def record_user_interests_details(email, phone, name="Name not provided", category="Not provided"):
    push_system_prompt = f"You are a push notification system. You are responsible for sending notifications to users about \
        upcoming events that they are interested in. Users interests and descriptions are in {category}. Please send a push notification \
        to the user with the event details."
    messages = [{"role":"user", "content": push_system_prompt}]

    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=messages
    )

    push_notification = response.choices[0].message.content
    push(push_notification)

    return {"pushed": "ok"}

In [100]:
globals()["record_user_interests_details"]("randhawa.harpreet2023@gmail.com")

Push: Recording interest from Name not provided with email randhawa.harpreet2023@gmail.com and interests Not provided


{'recorded': 'ok'}

In [137]:
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)

        if tool_name == "record_user_interests_details":
            result = record_user_interests_details(**arguments)
            global_name = json.dumps(result)
        
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [138]:
system_prompt = f"You are acting as event notification system. Ask users questions to tailor their interests and understand their preferences \
    subcategories. Ask them for their email and phone number including country code to send them notifications. End the conversation by \
    thanking them for their time. At the end of the conversation use your record_user_interests_details tool to record the the events the user wants to be \
    notified about and their contact information."

system_prompt += f"With this context, please chat with the user, always staying in character as an event notification system. Be the one to initiate the \
    converstation."


In [139]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    done = False
    while not done:
        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)
        finish_reason = response.choices[0].finish_reason

        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
            
    return response.choices[0].message.content

In [147]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7878
* To create a public link, set `share=True` in `launch()`.


In [ ]:
push(global_name)

Push: HEY
